In [5]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np


In [6]:
# import data.tsv file and create a dataframe

df = pd.read_csv('data.tsv', sep='\t')

# open file data_info.txt and read the first line

sequence_length = None
with open('data_info.txt', 'r') as f:
    line = f.readline()
    split_line_on_collen = line.split(':')
    sequence_length = int(split_line_on_collen[1])
    


train_df = df.sample(frac=0.8, random_state=0)
test_df = df.drop(train_df.index)


# drop the collumn Y from the dataframe

train_x = train_df.drop(['Y'], axis=1).iloc[1:]
train_y = train_df['Y']

# make the tran_x data rows have the same length as the sequence_length by padding with [0,0,0] for each collumn

# train_x = train_x.reindex(range(sequence_length), fill_value=[0,0,0])

test_x = test_df.drop(['Y'], axis=1)


test_y = test_df['Y']



In [7]:
def convert_rows_to_nupy_array(df):
    numpy_array = []
    for index, row in df.iterrows():
        row_array = []
        for col in row.iteritems():
            col_float = []
            for item in col[1].split(','):
                col_float.append(float(item.replace('[', '').replace(']', '')))
            row_array.append(col_float)
        numpy_array.append(row_array)
    return np.array(numpy_array)

train_x_numpy = convert_rows_to_nupy_array(train_x)
test_x_numpy = convert_rows_to_nupy_array(test_x)

print(train_x_numpy.shape)
print(test_x_numpy[1][5])
            

(2057, 749, 3)
[ 0.51171875 -0.10406494 -0.10406494]


##### Remove Padding from numpy

In [8]:
train_y_no_pad = train_y
test_y_no_pad = test_y

# remove the numpy array at the axis 3 that has values of a numpy array with all zeros

train_x_numpy_no_pad = np.delete(train_x_numpy, np.where(~train_x_numpy.any(axis=(0,1,2))), axis=0)
test_x_numpy_no_pad = np.delete(test_x_numpy, np.where(~test_x_numpy.any(axis=(0,1,2))), axis=0)
print(train_x_numpy_no_pad.shape)

[[[-3.4326172e-01  9.3945310e-01  9.3945310e-01]
  [-3.4326172e-01  9.3945310e-01  9.3945310e-01]
  [-3.4326172e-01  9.3945310e-01  9.3945310e-01]
  ...
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]]

 [[ 6.2613687e-03  2.1991149e-02  2.1991149e-02]
  [ 1.1148291e-02  2.0769420e-02  2.0769420e-02]
  [ 1.4813482e-02  2.3212880e-02  2.3212880e-02]
  ...
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]]

 [[ 2.2692871e-01 -3.9367676e-02 -3.9367676e-02]
  [ 2.2692871e-01 -3.9367676e-02 -3.9367676e-02]
  [ 2.2692871e-01 -3.9367676e-02 -3.9367676e-02]
  ...
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]]

 ...

 [[ 2.9016100e-03  3.0543262e-04  3.0543262e-04]
  [ 4.5814895e-04  3.0543262e-04  3.054

## Train

In [9]:
# Dummy Classifer

from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(train_x_numpy, train_y)
y_pred = dummy_clf.predict(test_x_numpy)
print(accuracy_score(test_y, y_pred))

ValueError: Found input variables with inconsistent numbers of samples: [2057, 2058]

In [ ]:

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, input,target, seq_len):
        self.input = input
        self.target = target
        self.seq_len = seq_len
    def __getitem__(self, item):
        # print("input item", self.input[item])
        return self.input[item], self.target.iloc[item]

        # return input[item:item+self.seq_len], input[item+self.seq_len]
    def __len__(self):
        return self.input.shape[0]

# test_input = np.arange(1,8).reshape(-1,1)
# input = torch.tensor(test_input, dtype=torch.float)

# dataset = MyDataset(input, 3)

# dl = torch.utils.data.DataLoader(dataset, batch_size=1)
# for inp, label in dl:
#     print("inp shape",inp.shape)
#     print("inp",inp.numpy())


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


input_size = 3
num_classes = 2
hidden_size = 256
num_layers = 2
learning_rate = 0.001

class SitUpDetector(nn.Module):
    def __init__(self,input_size, num_classes, hidden_size, num_layers):
        super(SitUpDetector, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out
    

model = SitUpDetector(input_size, num_classes, hidden_size, num_layers).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


myDataset = MyDataset(train_x_numpy,train_y, sequence_length)
train_loader = torch.utils.data.DataLoader(dataset=myDataset, batch_size=1)


num_epochs = 5
total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (features, target) in enumerate(train_loader):

        # convert Double tensor to Float tensor
        features = features.float()

        outputs = model(features)

        loss = criterion(outputs, target)
        optimizer.zero_grad()
        
        loss.backward()

        optimizer.step()
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_steps, loss.item()))




        
        

# test the model

with torch.no_grad():
    output = model(test_x)
    print(output)
    print(test_y)


# save the model

torch.save(model.state_dict(), 'sit_up_detector.pth')






Epoch [1/100], Step [100/2057], Loss: 0.1597
Epoch [1/100], Step [200/2057], Loss: 1.3734
Epoch [1/100], Step [300/2057], Loss: 0.2067
Epoch [1/100], Step [400/2057], Loss: 1.2511
Epoch [1/100], Step [500/2057], Loss: 0.2528
Epoch [1/100], Step [600/2057], Loss: 0.2503
Epoch [1/100], Step [700/2057], Loss: 0.2768
Epoch [1/100], Step [800/2057], Loss: 1.2031
Epoch [1/100], Step [900/2057], Loss: 0.2962
Epoch [1/100], Step [1000/2057], Loss: 0.2959
Epoch [1/100], Step [1100/2057], Loss: 0.2957
Epoch [1/100], Step [1200/2057], Loss: 0.3633
Epoch [1/100], Step [1300/2057], Loss: 0.3614
Epoch [1/100], Step [1400/2057], Loss: 0.3259
Epoch [1/100], Step [1500/2057], Loss: 0.3964
Epoch [1/100], Step [1600/2057], Loss: 0.2617
Epoch [1/100], Step [1700/2057], Loss: 1.3364
Epoch [1/100], Step [1800/2057], Loss: 1.4455
Epoch [1/100], Step [1900/2057], Loss: 1.1420
Epoch [1/100], Step [2000/2057], Loss: 1.1775
Epoch [2/100], Step [100/2057], Loss: 0.2431
Epoch [2/100], Step [200/2057], Loss: 1.4006

KeyboardInterrupt: 

# Train with no Padding

In [ ]:
# Make a new Pytorch LSTM model that takes a variable input

class SitUpDetectorVariable(nn.Module):
    def __init__(self,input_size, num_classes, hidden_size, num_layers):
        super(SitUpDetectorVariable, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# train the model

model = SitUpDetectorVariable(input_size, num_classes, hidden_size, num_layers).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

myDataset = MyDataset(train_x_numpy_no_pad,train_y, sequence_length)
train_loader = torch.utils.data.DataLoader(dataset=myDataset, batch_size=1)

num_epochs = 5
total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (features, target) in enumerate(train_loader):

        # convert Double tensor to Float tensor
        features = features.float()

        outputs = model(features)

        loss = criterion(outputs, target)
        optimizer.zero_grad()
        
        loss.backward()

        optimizer.step()
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_steps, loss.item()))
                   